In [258]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Sellers

Our goal is to find Sellers that repeatedly underperform vs others, and understand why.  
This will help us shape our recommendations on how to improve Olist's profit margin

<details>
    <summary>🔥 Notebook best practices (reminder) </summary>

From now on, exploratory notebooks are going to get pretty long, and we strongly advise you to follow these notebook principles
- Code your logic so that your Notebook can always be run from top to bottom without crashing (Cell --> Run All)
- Name your variables carefully 
- Use dummy names such as `tmp` for intermediary steps when you know you won't need them long
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Hide your cell output if you don't need to see it anymore (double click on the red `Out[]:` section to the left of your cell).
- Make heavy use of jupyter nbextention `Collapsable Headings` and `Table of Content` (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you're between method brackets e.g. `group_by()` to get the docs! Repeat a few times to open it permanently

</details>





## 1 - Code `olist/seller.py` 

- Create the method `get_training_data` in `olist/seller.py` that will return the following DataFrame:

  - `seller_id` (_str_) _the id of the seller **UNIQUE**
  - `seller_city` (_str_) _the city where seller is located_
  - `seller_state` (_str_) _the state where seller is located_
  - `delay_to_carrier` (_float_) _Average delay_to_carrier per seller. Return 0 if the order is delivered before the shipping_limit_date, otherwise the absolute value of the delay
  - `wait_time` (_float_) _Average wait_time (duration of deliveries) per seller_
  - `date_first_sale` (_datetime_) _Date of first sales on Olist_
  - `date_last_sale` (_datetime_) _Date of last sales on Olist_
  - `share_of_five_stars` (_float_) _The share of five star orders for orders in which the seller was involved_
  - `share_of_one_stars` (_float_) _The share of one star orders for orders in which the seller was involved_
  - `review_score` (_float_) _The average review score for orders in which the seller was involved_
  - `n_orders` (_int_) _The number of unique orders the seller was involved with._
  - `quantity` (_int_) _The total number of items sold by this seller_
  - `quantity_per_order`(_float_) _The mean number of items per order for this seller_
  - `sales` (_float_) _The total sales associated with this seller (excluding freight value)_ in BRL
  
Feel free to code all intermediary methods below if you prefer to breakdown the problem step by step.

In [259]:
from olist.data import Olist
olist=Olist()
data=olist.get_data()

### `get_seller_features`
Returns a DataFrame with: 'seller_id', 'seller_city', 'seller_state'

In [98]:
sellers = data['sellers'].copy()
print(sellers.shape)
print(sellers.columns)
sellers_n = sellers[['seller_id', 'seller_city', 'seller_state']]
print(sellers_n.shape)

(3095, 4)
Index(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'], dtype='object')
(3095, 3)


In [4]:
# sellers_wagon_solution
sellers_wagon = data['sellers'].copy()
sellers_wagon.drop('seller_zip_code_prefix', axis=1, inplace=True)
# There are multiple rows per seller
sellers_wagon.drop_duplicates(inplace=True)
print(sellers_wagon.shape)

(3095, 3)


### `get_seller_delay_wait_time`
Returns a DataFrame with: 'seller_id', 'delay_to_carrier', 'wait_time'

In [5]:
#delay_to_carrier (float) _Average delay_to_carrier per seller. 
#Return 0 if the order is delivered before the shipping_limit_date, otherwise the absolute value of the delay
#wait_time (float) Average wait_time (duration of deliveries) per seller

In [99]:
sellers.head(1)

seller_id  seller_zip_code_prefix seller_city  \
0  3442f8959a84dea7ee197c632cb2df15                   13023    campinas   

  seller_state  
0           SP

In [260]:
# compute wait time
orders = data['orders'].copy()
print(orders.shape)

(99441, 8)


In [261]:
delivered_orders = orders[orders['order_status'] == 'delivered'].copy()
print(delivered_orders.shape)
delivered_orders.head(1)

(96478, 8)


order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   

  order_estimated_delivery_date  
0           2017-10-18 00:00:00

In [262]:
order_item = data['order_items'].copy()
print(order_item.shape)
order_item.head(1)

(112650, 7)


order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   

   shipping_limit_date  price  freight_value  
0  2017-09-19 09:45:35   58.9          13.29

In [263]:
order_seller = order_item[['order_id', 'seller_id', 'shipping_limit_date']] \
.merge( delivered_orders[['order_id', 'order_purchase_timestamp',
                                 'order_delivered_carrier_date', 'order_delivered_customer_date']] \
        , on = "order_id")  
print(order_seller.shape)
order_seller.head(1)

(110197, 6)


order_id                         seller_id  \
0  00010242fe8c5a6d1ba2dd792cb16214  48436dade18ac8b2bce089ec2a041202   

   shipping_limit_date order_purchase_timestamp order_delivered_carrier_date  \
0  2017-09-19 09:45:35      2017-09-13 08:59:02          2017-09-19 18:34:16   

  order_delivered_customer_date  
0           2017-09-20 23:43:48

In [264]:
order_seller.loc[:, 'order_purchase_timestamp'] = pd.to_datetime(
    order_seller['order_purchase_timestamp'], format='%Y-%m-%d')

order_seller.loc[:,'order_delivered_customer_date'] = pd.to_datetime(
    order_seller['order_delivered_customer_date'], format='%Y-%m-%d')

order_seller.loc[:,'order_delivered_carrier_date'] = pd.to_datetime(
    order_seller['order_delivered_carrier_date'], format='%Y-%m-%d')

order_seller.loc[:,'shipping_limit_date'] = pd.to_datetime(
    order_seller['shipping_limit_date'], format='%Y-%m-%d')

In [265]:
#Return 0 if the order is delivered before the shipping_limit_date, otherwise the absolute value of the delay

def negative_date(d):
    if d < 0:
        return abs(d)
    return 0

In [266]:
def order_wait_time(df):
            days = np.mean(
                (df.order_delivered_customer_date - df.order_purchase_timestamp)
                / np.timedelta64(24, 'h'))
            return days
        
def delay_to_logistic_partner(df):
            df['delay'] = (df.shipping_limit_date -
                      df.order_delivered_carrier_date) / np.timedelta64(24, 'h')
            df.loc[:,'delay'] = df.delay.apply(negative_date)
            return np.mean(df.delay)


In [267]:
order_seller_0 = order_seller[['seller_id', 'order_delivered_customer_date', 
                             'order_delivered_carrier_date',
                               'shipping_limit_date', 'order_purchase_timestamp']].copy()
order_seller_0.head(7)

seller_id order_delivered_customer_date  \
0  48436dade18ac8b2bce089ec2a041202           2017-09-20 23:43:48   
1  dd7ddc04e1b6c2c614352b383efe2d36           2017-05-12 16:04:24   
2  5b51032eddd242adc84c38acab88f23d           2018-01-22 13:19:16   
3  9d7a1d34a5052409006425275ba1c2b4           2018-08-14 13:32:39   
4  df560393f3a51e74553ab94004ba5c87           2017-03-01 16:42:31   
5  6426d21aca402a131fc0a5d0960a3c90           2017-05-22 13:44:35   
6  7040e82f899a04d1b434b795a43b4617           2017-12-18 22:03:38   

  order_delivered_carrier_date shipping_limit_date order_purchase_timestamp  
0          2017-09-19 18:34:16 2017-09-19 09:45:35      2017-09-13 08:59:02  
1          2017-05-04 14:35:00 2017-05-03 11:05:13      2017-04-26 10:53:06  
2          2018-01-16 12:36:48 2018-01-18 14:48:30      2018-01-14 14:33:31  
3          2018-08-10 13:28:00 2018-08-15 10:10:18      2018-08-08 10:00:35  
4          2017-02-16 09:46:09 2017-02-13 13:57:51      2017-02-04 13:57:51  
5          2017-05-17 11:05:55 2017-05-23 03:55:27      2017-05-15 21:42:34  
6          2017-12-12 01:07:48 2017-12-14 12:10:31      2017-12-10 11:53:48

In [268]:
wait_time = order_seller_0.groupby('seller_id')\
.apply( order_wait_time
     )  \
    .reset_index()
wait_time.columns = ['seller_id', 'wait_time']
wait_time.head(2)

seller_id  wait_time
0  0015a82c2db000af6aaaf3ae2ecb0532  10.793885
1  001cca7ae9ae17fb1caed9dfb1094831  13.096632

In [269]:
#Return 0 if the order is delivered before the shipping_limit_date, otherwise the absolute value of the delay

delay_time = order_seller_0.groupby('seller_id')\
.apply(delay_to_logistic_partner)\
    .reset_index()
delay_time.columns = ['seller_id', 'delay_to_carrier']

delay_time.head(2)

seller_id  delay_to_carrier
0  0015a82c2db000af6aaaf3ae2ecb0532           0.00000
1  001cca7ae9ae17fb1caed9dfb1094831           0.06326

In [270]:
order_delay_wait_time = delay_time.merge(wait_time,  on='seller_id')
#order_delay_wait_time.columns = ['seller_id', 'delay_to_carrier', 'wait_time']

In [271]:
order_delay_wait_time .head(2)

seller_id  delay_to_carrier  wait_time
0  0015a82c2db000af6aaaf3ae2ecb0532           0.00000  10.793885
1  001cca7ae9ae17fb1caed9dfb1094831           0.06326  13.096632

In [255]:
order_delay_wait_time.delay_to_carrier.min()

0.0

In [256]:
order_delay_wait_time.wait_time.min()

1.2141782407407407

In [272]:
order_delay_wait_time.wait_time.max()

189.86315972222224

### `get_active_dates`
Returns a DataFrame with 'seller_id', 'date_first_sale', 'date_last_sale'

In [198]:
order_seller.head(2)

order_id order_purchase_timestamp  \
0  e481f51cbdc54678b7cc49136f2d6af7      2017-10-02 10:56:33   
1  53cdb2fc8bc7dce0b6741e2150273451      2018-07-24 20:41:37   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00           2018-08-07 15:27:45   

                          seller_id shipping_limit_date       wait  delay  
0  3504c0cb71d7fa48d967e0e4c94d59d9 2017-10-06 11:07:15   8.436574    0.0  
1  289cdb325fb7e7f891c38608bf9e0962 2018-07-30 03:24:27  13.782037    0.0

In [199]:
activ_dates = order_seller[ ['order_id','seller_id',
                                                    'order_purchase_timestamp' ]] 

print(activ_dates.shape)   
activ_dates.head(2)

(110197, 3)


order_id                         seller_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  3504c0cb71d7fa48d967e0e4c94d59d9   
1  53cdb2fc8bc7dce0b6741e2150273451  289cdb325fb7e7f891c38608bf9e0962   

  order_purchase_timestamp  
0      2017-10-02 10:56:33  
1      2018-07-24 20:41:37

In [200]:
(activ_dates['seller_id'].unique()).shape

(2970,)

In [201]:
(activ_dates['order_id'].unique()).shape

(96478,)

In [202]:

activ_dates.groupby('seller_id',as_index=False).agg({'order_purchase_timestamp': [np.min,np.max]}) \
.rename(columns={'amin': "date_first_sale", 'amax' : 'date_last_sale'}).columns


MultiIndex([(               'seller_id',                ''),
            ('order_purchase_timestamp', 'date_first_sale'),
            ('order_purchase_timestamp',  'date_last_sale')],
           )

In [204]:

activ_dates.groupby('seller_id',as_index=False) \
           .agg({'order_purchase_timestamp': [np.min,np.max]}) \


activ_dates.columns= ["seller_id", "date_first_sale", 'date_last_sale'] 
activ_dates.head(2)


seller_id                   date_first_sale  \
0  e481f51cbdc54678b7cc49136f2d6af7  3504c0cb71d7fa48d967e0e4c94d59d9   
1  53cdb2fc8bc7dce0b6741e2150273451  289cdb325fb7e7f891c38608bf9e0962   

       date_last_sale  
0 2017-10-02 10:56:33  
1 2018-07-24 20:41:37

In [45]:
# active date _wagon solution

# Import olist data
from olist.data import Olist
olist=Olist()
data=olist.get_data()
matching_table = olist.get_matching_table()
matching_table.shape

(114100, 5)

In [46]:
# active date _wagon solution
orders_wagon = data['orders'][['order_id', 'order_approved_at']].copy()
print(orders_wagon.shape)
# create two new columns with a view to aggregate
orders_wagon.loc[:, 'date_first_sale'] = pd.to_datetime(
            orders_wagon['order_approved_at'])
orders_wagon['date_last_sale'] = orders_wagon['date_first_sale']
orders_wagon.head(2)

(99441, 2)


order_id    order_approved_at     date_first_sale  \
0  e481f51cbdc54678b7cc49136f2d6af7  2017-10-02 11:07:15 2017-10-02 11:07:15   
1  53cdb2fc8bc7dce0b6741e2150273451  2018-07-26 03:24:27 2018-07-26 03:24:27   

       date_last_sale  
0 2017-10-02 11:07:15  
1 2018-07-26 03:24:27

In [47]:
# active date _wagon solution
orders_wagon.merge(
            matching_table[['seller_id', 'order_id']], on="order_id")\
            .groupby('seller_id')\
            .agg({
                "date_first_sale": min,
                "date_last_sale": max
            })
print(orders_wagon.shape)
orders_wagon.head(3)

(99441, 4)


order_id    order_approved_at     date_first_sale  \
0  e481f51cbdc54678b7cc49136f2d6af7  2017-10-02 11:07:15 2017-10-02 11:07:15   
1  53cdb2fc8bc7dce0b6741e2150273451  2018-07-26 03:24:27 2018-07-26 03:24:27   
2  47770eb9100c2d0c44946d9cf07ec65d  2018-08-08 08:55:23 2018-08-08 08:55:23   

       date_last_sale  
0 2017-10-02 11:07:15  
1 2018-07-26 03:24:27  
2 2018-08-08 08:55:23

### `get_review_score`
['seller_id', 'share_of_five_stars', 'share_of_one_stars', 'review_score']

In [4]:
order_item = data['order_items'].copy(); print(order_item.shape)
orders = data['orders'].copy(); print(orders.shape)
order_reviews = data['order_reviews'].copy(); print(order_reviews.shape)

(112650, 7)
(99441, 8)
(100000, 7)


In [5]:
mask_columns = [ 'order_id', 'review_score','review_id' ]

merged = orders.merge(order_reviews, on = 'order_id')[mask_columns]
print(merged.shape)
merged.head(2)

(100000, 3)


order_id  review_score  \
0  e481f51cbdc54678b7cc49136f2d6af7             4   
1  53cdb2fc8bc7dce0b6741e2150273451             4   

                          review_id  
0  a54f0611adc9ed256b57ede6b6eb5114  
1  8d5266042046a06655c8db133d120ba5

In [6]:
order_item.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

In [7]:
mask_columns = [ 'order_id', 'seller_id', 
                'review_score','review_id' ]

merged = merged.merge(order_item, on = 'order_id')[mask_columns]
print(merged.shape)
merged.head(4)

(113322, 4)


order_id                         seller_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  3504c0cb71d7fa48d967e0e4c94d59d9   
1  53cdb2fc8bc7dce0b6741e2150273451  289cdb325fb7e7f891c38608bf9e0962   
2  47770eb9100c2d0c44946d9cf07ec65d  4869f7a5dfa277a7dca6462dcf3b52b2   
3  949d5b44dbf5de918fe9c16f97b45f8a  66922902710d126a0e7d26b0e3805106   

   review_score                         review_id  
0             4  a54f0611adc9ed256b57ede6b6eb5114  
1             4  8d5266042046a06655c8db133d120ba5  
2             5  e73b67b67587f7644d5bd1a52deb1b01  
3             5  359d03e676b3c069f62cadba8dd3f6e8

In [8]:
def dim_five_star(d):
    if d == 5:
        return 1
    return 0

def dim_one_star(d):
    if d == 1:
        return 1
    return 0

merged.loc[:, 'dim_is_five_star'] =\
            merged['review_score'].apply(dim_five_star)

merged.loc[:, 'dim_is_one_star'] =\
            merged['review_score'].apply(dim_one_star)
merged.head(4)


order_id                         seller_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  3504c0cb71d7fa48d967e0e4c94d59d9   
1  53cdb2fc8bc7dce0b6741e2150273451  289cdb325fb7e7f891c38608bf9e0962   
2  47770eb9100c2d0c44946d9cf07ec65d  4869f7a5dfa277a7dca6462dcf3b52b2   
3  949d5b44dbf5de918fe9c16f97b45f8a  66922902710d126a0e7d26b0e3805106   

   review_score                         review_id  dim_is_five_star  \
0             4  a54f0611adc9ed256b57ede6b6eb5114                 0   
1             4  8d5266042046a06655c8db133d120ba5                 0   
2             5  e73b67b67587f7644d5bd1a52deb1b01                 1   
3             5  359d03e676b3c069f62cadba8dd3f6e8                 1   

   dim_is_one_star  
0                0  
1                0  
2                0  
3                0

In [9]:
merged_score = merged.groupby(
            'seller_id', as_index=False).agg({'dim_is_five_star': 'mean',
                                              'dim_is_one_star': 'mean',
                                              'review_score': 'mean'})\
.rename(columns= {'dim_is_one_star' : 'share_of_one_stars',
                 'dim_is_five_star': 'share_of_five_stars'} )
merged_score.head(3)
# Rename colu

seller_id  share_of_five_stars  share_of_one_stars  \
0  0015a82c2db000af6aaaf3ae2ecb0532             0.666667            0.333333   
1  001cca7ae9ae17fb1caed9dfb1094831             0.506276            0.150628   
2  001e6ad469a905060d959994f1b41e4f             0.000000            1.000000   

   review_score  
0      3.666667  
1      3.874477  
2      1.000000

In [11]:
merged_score['review_score'].mean()

3.956892695448315

In [10]:
## review_score wagon solution

from olist.order import Order
order = Order()

In [75]:
## review_score wagon solution
orders_reviews = order.get_review_score()

# Since the same seller can appear multiple times in the same order,
# create a (seller <> order) matching table

matching_table = matching_table[['order_id', 'seller_id']]\
            .drop_duplicates()
reviews_df = matching_table.merge(orders_reviews, on='order_id')
reviews_df = reviews_df.groupby(
            'seller_id', as_index=False).agg({'dim_is_one_star': 'mean',
                                              'dim_is_five_star': 'mean',
                                              'review_score': 'mean'})
# Rename columns
reviews_df.columns = ['seller_id', 'share_of_five_stars',
                              'share_of_one_stars', 'review_score']

reviews_df.head(3)

seller_id  share_of_one_stars  share_of_five_stars  \
0  0015a82c2db000af6aaaf3ae2ecb0532            0.333333             0.666667   
1  001cca7ae9ae17fb1caed9dfb1094831            0.130000             0.520000   
2  001e6ad469a905060d959994f1b41e4f            1.000000             0.000000   

   review_score  
0      3.666667  
1      3.950000  
2      1.000000

### `get_quantity`
['seller_id', 'n_orders', 'quantity', 'quantity_per_order']

In [ ]:
#quantity (int) The total number of items sold by this seller
#quantity_per_order(float) The mean number of items per order for this seller
# n_orders (int) The number of unique orders the seller was involved with.

In [174]:
order_item = data['order_items'].copy(); print(order_item.shape)
order_item.head(3)

(112650, 7)


order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d   

   shipping_limit_date  price  freight_value  
0  2017-09-19 09:45:35   58.9          13.29  
1  2017-05-03 11:05:13  239.9          19.93  
2  2018-01-18 14:48:30  199.0          17.87

In [167]:
order_item.groupby('seller_id', as_index= False)['order_id'].count()\
        .rename(columns={"order_id" : "n_orders"} ) .head(3) # we don't consider 
                                                         #this cell because some rows duplicate -->nunique()

seller_id  n_orders
0  0015a82c2db000af6aaaf3ae2ecb0532         3
1  001cca7ae9ae17fb1caed9dfb1094831       239
2  001e6ad469a905060d959994f1b41e4f         1

In [175]:
quantity_unique = order_item.groupby('seller_id', as_index= False)['order_id'].nunique()\
       .rename(columns={"order_id" : "n_orders"} )
quantity_unique.head(3)

seller_id  n_orders
0  0015a82c2db000af6aaaf3ae2ecb0532         3
1  001cca7ae9ae17fb1caed9dfb1094831       200
2  001e6ad469a905060d959994f1b41e4f         1

In [176]:
quantity_total = order_item.groupby('seller_id', as_index= False)['order_id'].count()\
        .rename(columns={"order_id" : "quantity"} ) 
quantity_total.head(3)

seller_id  quantity
0  0015a82c2db000af6aaaf3ae2ecb0532         3
1  001cca7ae9ae17fb1caed9dfb1094831       239
2  001e6ad469a905060d959994f1b41e4f         1

In [177]:
#quantity_per_order(float) The mean number of items per order for this seller

quantity_res = quantity_unique.merge(quantity_total, on = "seller_id")
quantity_res.shape

(3095, 3)

In [178]:
quantity_res['quantity_per_order'] = quantity_res['n_orders'] / quantity_res['quantity']
quantity_res.head(3)

seller_id  n_orders  quantity  quantity_per_order
0  0015a82c2db000af6aaaf3ae2ecb0532         3         3             1.00000
1  001cca7ae9ae17fb1caed9dfb1094831       200       239             0.83682
2  001e6ad469a905060d959994f1b41e4f         1         1             1.00000

### `get_sales`
['seller_id', 'sales']

In [25]:
#`sales` (_float_) _The total sales associated with this seller (excluding freight value)_ in BRL

order_item = data['order_items'].copy(); print(order_item.shape)
order_item.head(3)

(112650, 7)


order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   

                         product_id                         seller_id  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d   

   shipping_limit_date  price  freight_value  
0  2017-09-19 09:45:35   58.9          13.29  
1  2017-05-03 11:05:13  239.9          19.93  
2  2018-01-18 14:48:30  199.0          17.87

In [26]:
order_item = order_item[['order_id', 'seller_id', 'price']]
order_item.head(3)

order_id                         seller_id  price
0  00010242fe8c5a6d1ba2dd792cb16214  48436dade18ac8b2bce089ec2a041202   58.9
1  00018f77f2f0320c557190d7a144bdd3  dd7ddc04e1b6c2c614352b383efe2d36  239.9
2  000229ec398224ef6ca0657da4fc703e  5b51032eddd242adc84c38acab88f23d  199.0

In [27]:
order_item = order_item.groupby('seller_id', as_index= False)['price'].agg(["sum"]).reset_index() \
.rename(columns={'sum': 'sales'}) 
order_item.head(3)

seller_id     sales
0  0015a82c2db000af6aaaf3ae2ecb0532   2685.00
1  001cca7ae9ae17fb1caed9dfb1094831  25080.03
2  001e6ad469a905060d959994f1b41e4f    250.00

### Save your logic into seller.py

### Test your code

In [5]:
from nbresult import ChallengeResult
from olist.seller import Seller

sellers = Seller().get_training_data()
sellers.head()
data = Seller().get_training_data()
result = ChallengeResult('seller',
shape=data.shape,
columns=sorted(list(data.columns)),
avg_review_score=int(data['review_score'].mean()),
unique_state=sorted(list(data['seller_state'].unique())),
min_wait_time=round(data['wait_time'].min(), 2),
max_wait_time=int(data['wait_time'].max()),
avg_wait_time=int(data['wait_time'].mean()),
avg_delay_carrier=round(data['delay_to_carrier'].mean(), 2),
avg_quantity=int(data['quantity'].mean()),
max_quantity=data['quantity'].max(),
min_quantity=data['quantity'].min(),
avg_sales=int(data['sales'].mean())
)
result.write()
print(result.check())

****** 3.982832942865022
******                           seller_id seller_city seller_state  \
0  3442f8959a84dea7ee197c632cb2df15    campinas           SP   

   delay_to_carrier  wait_time     date_first_sale      date_last_sale  
0          1.514329  13.018588 2017-05-05 16:25:11 2017-08-30 12:50:19  
***score***                           seller_id seller_city seller_state  \
0  3442f8959a84dea7ee197c632cb2df15    campinas           SP   

   delay_to_carrier  wait_time     date_first_sale      date_last_sale  \
0          1.514329  13.018588 2017-05-05 16:25:11 2017-08-30 12:50:19   

   share_of_one_stars  share_of_five_stars  review_score  
0            0.333333             0.333333           3.0  
***quantity***                           seller_id seller_city seller_state  \
0  3442f8959a84dea7ee197c632cb2df15    campinas           SP   

   delay_to_carrier  wait_time     date_first_sale      date_last_sale  \
0          1.514329  13.018588 2017-05-05 16:25:11 2017-08-30 12:50

In [275]:
from nbresult import ChallengeResult
from olist.my_seller import Seller

sellers = Seller().get_training_data()
sellers.head()
data = Seller().get_training_data()
result = ChallengeResult('seller',
shape=data.shape,
columns=sorted(list(data.columns)),
avg_review_score=int(data['review_score'].mean()),
unique_state=sorted(list(data['seller_state'].unique())),
min_wait_time=round(data['wait_time'].min(), 2),
max_wait_time=int(data['wait_time'].max()),
avg_wait_time=int(data['wait_time'].mean()),
avg_delay_carrier=round(data['delay_to_carrier'].mean(), 2),
avg_quantity=int(data['quantity'].mean()),
max_quantity=data['quantity'].max(),
min_quantity=data['quantity'].min(),
avg_sales=int(data['sales'].mean())
)
result.write()
print(result.check())

test______> 1.2141782407407407
test______> 1.2141782407407407
============================= test session starts ==============================
platform darwin -- Python 3.8.6, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /Users/kenzaelhoussaini/.pyenv/versions/3.8.6/bin/python3
cachedir: .pytest_cache
rootdir: /Users/kenzaelhoussaini/code/kelhoussaini/data-challenges/04-Decision-Science/03-Linear-Regression/02-Sellers
plugins: dash-1.20.0, anyio-3.2.1
collecting ... collected 8 items

tests/test_seller.py::TestSeller::test_average_delay_carrier PASSED      [ 12%]
tests/test_seller.py::TestSeller::test_average_review_score PASSED       [ 25%]
tests/test_seller.py::TestSeller::test_average_sales PASSED              [ 37%]
tests/test_seller.py::TestSeller::test_columns PASSED                    [ 50%]
tests/test_seller.py::TestSeller::test_quantity PASSED                   [ 62%]
tests/test_seller.py::TestSeller::test_shape PASSED                      [ 75%]
tests/test_seller.py::TestSeller:

✅ Once your logic is encoded, commit and push your new file `seller.py`  
✅ Feel free to continue working on this notebook, or make a copy and delete the code above to start section _2 - Exploration_ from a fresh noteook

## 2 - Exploration

### 2.1 - Plots

❓ Let's start with some initial exploratory analysis on sellers distribution:

- Plot the distribution of each numerical variable of the dataset in one large figure
- Do you notice any outliers?
- What's the median of orders per seller? How is the distribution of that variable looking?

In [ ]:
# Your code

----
💡There seems to be a group of sellers which stands out for having very low review scores! Let's investigate graphically:

❓ Using plotly, create a scatterplot of `delay_to_carrier` against `wait_time`, varying bubble size by total `sales` for that seller, and coloring by `review_score`. 

In [ ]:
import plotly.express as px

Feel free to change values `x`, `y`, `color` and `size` to try identify who are the worst sellers

### 2.2 - Model out `review_score` with OLS

❓ Scatter plots have their limits. A more rigorous way to explain sellers' review_score is to **model out the impact of various features on `review_score` with a multivariate-OLS in statsmodels.** 

Create an OLS with only the numerical features of your choice. What are the most impactful ones? Don't forget to standardize your features using our function below if you want to compare the regression coefficients together.

In [ ]:
def standardize(df, features):
    df_standardized = df.copy()
    for f in features:
        mu = df[f].mean()
        sigma = df[f].std()
        df_standardized[f] = df[f].map(lambda x: (x - mu) / sigma)
    return df_standardized

In [ ]:
# Your code

❓ Finally, investigate your model's performance (r-squared) and residuals

In [ ]:
# Your code

### 2.3 - Add seller_state to your analysis

❓ We haven't used information about `seller_state` yet.  
- Create a new OLS model regressing `review_score` on `seller_states` only.
- Analyse your significant features using `return_significative_coef(model)` coded for you in `olist/utils.py`
- What are the best states in terms of `review_score`? 

<details>
    <summary>Hints</summary>
        
Be careful, seller_state is a categorical feature. Use `C(a_cat_feature)` in the formula to indicate categorical variables. It will create one boolean variable `is_cat_feature_xx` **per unique category** 

</details>

In [ ]:
# Your code

☝️ Some states indeed have _signigicantly_ better reviews than others on average. Is it due to some lower `quantity_per_order`, lower `wait_time`, or `delay_to_carrier`? Or is it due to some other factors that we haven't collected data about?

❓ Try to isolate the impact of the `seller_state` from the rest by adding other continuous features to your OLS until seller_states is no longer statistically siginificant.

In [ ]:
# Your code

☝️ After adding `wait_time` to our analysis, none of the 22 dummy variables `is_seller_state_xx` are statistically signigicant:

Given our small dataset (most states have very few sellers):
- We _cannot conclude_ that "some states are inherently better than other for reasons that would be independent of the `wait_time`" 
- In other words, we _cannot reject the hypothesis_ that "seller_state has no impact on review_score, other than through `wait_time`"

🏁 **Congratulations! Commit and push your notebook as well as `seller.py`**